In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataframe_image as dfi

##https://apps.bea.gov/national/Release/XLS/Survey/Section1All_xls.xlsx

df=pd.read_excel('Section1All_xls.xlsx','T11500-Q',skiprows=7).query('Line in ["1","2","3","7"]').iloc[:,-20:]
df.index=['Prices','Labor','Other Inputs','Profits']
df2=df.transpose()
df2.index=pd.to_datetime(list(df2.index))

df2

/var/folders/3c/6cgp3y4n5svbnrtm237wj5680000gn/T/ipykernel_27446/3261998057.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2.index=pd.to_datetime(list(df2.index))


,Prices,Labor,Other Inputs,Profits
2019-07-01,1.044,0.609,0.297,0.138
2019-10-01,1.045,0.612,0.293,0.140
2020-01-01,1.044,0.624,0.295,0.124
2020-04-01,1.042,0.651,0.262,0.129
2020-07-01,1.067,0.641,0.245,0.182
2020-10-01,1.063,0.635,0.275,0.153
2021-01-01,1.078,0.629,0.279,0.171
2021-04-01,1.100,0.639,0.270,0.192
2021-07-01,1.110,0.648,0.279,0.183
2021-10-01,1.131,0.659,0.292,0.181


In [8]:
df_delta=df2.iloc[-1]-df2
df_delta = df_delta[:-1]

df_out=df_delta.divide(df_delta.Prices,axis=0)[['Labor','Other Inputs','Profits']]
df_out=df_out[['Profits','Labor','Other Inputs']]

def make_pretty(styler):
    styler.set_caption("Share of Price Changes (Value Added) Through Q2 2024")\
    .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: top; font-size:1.25em;'
    }], overwrite=False)
    #styler.background_gradient()
    styler.format(precision=1)
    return styler

df_table= pd.concat([df_out[(df_out.index.month==4) & (df_out.index.year < 2024)], pd.DataFrame(df_out.iloc[[-2]])],axis=0).transpose().copy()*100
df_table.columns=['Since 2020','Since 2021','Since 2022','Since 2023','Last 2 Quarters']

# beginning of year
df_table3= pd.concat([df_out[(df_out.index.month==1) & (df_out.index.year < 2024)], pd.DataFrame(df_out.iloc[[-2]])],axis=0).transpose().copy()*100
df_table3.columns=['Last 4 Years','Last 3 Years','Last 2 Years','Last Year','Last 2 Quarters']

dfi.export(df_table.style.pipe(make_pretty), 'df_beginning_of_year.png',dpi=200)
dfi.export(df_table3.style.pipe(make_pretty), 'df_annual.png',dpi=200)


df_table.style.pipe(make_pretty)

,Since 2020,Since 2021,Since 2022,Since 2023,Last 2 Quarters
Profits,39.1,5.8,-12.9,55.6,-225.0
Labor,30.7,55.4,103.2,44.4,225.0
Other Inputs,30.2,38.0,6.5,0.0,100.0
